In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 1
from imports import *
from functions import *
from database import *

# Get Available Stations ID List

In [3]:
# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()
# parse the response as a dictionary
stations_df = api.stations(as_df=True)

print(len(stations_df))

146


# Filter Buoys by Remarks

In [4]:
access_error_url_list = []

# Liste de mots à rechercher dans la colonne "Remark"
blacklist = ["Failure", "ceased", "failed", "recovered", "stopped", 'adrift']
stations_id_set = set()

print(f'Avant Filtre: {stations_df.shape[0]}')

# Liste pour collecter les indices à supprimer
indices_a_supprimer = []

# Parcours des lignes de la DataFrame
for idx, row in stations_df.iterrows():
    station_id = row["Station"]
    station_Location = row["Hull No./Config and Location"]  # Extraire la valeur de la cellule pour chaque ligne
    
    # Extraction du nom de la station si un ")" est trouvé
    if ")" in station_Location:
        station_name = station_Location.split(')')[1].rstrip(" )")  # On enlève l'espace et la parenthèse en fin de chaîne
    else:
        station_name = station_Location.strip()  # Si pas de ")", on garde toute la chaîne

    station_name = station_name.rstrip(" )").replace("(", "").replace(")", "").strip()

    # Nettoyage final pour enlever toute parenthèse ou espace en fin de nom
    station_name = station_name.rstrip(" )")

    # Vérifier si "Remark" n'est pas NaN et si un des éléments de blacklist est dans "Remark"
    if isinstance(row["Remark"], str) and any(blacklist_word.lower() in row["Remark"].lower() for blacklist_word in blacklist):
        # Ajouter l'index à la liste
        indices_a_supprimer.append(idx)
        url = get_buoy_url(station_id)
        access_error_url_list.append(url)
    else:
        pass
# Supprimer les lignes après la boucle
stations_df.drop(index=indices_a_supprimer, inplace=True)

print(f'Après Filtre: {stations_df.shape[0]}')

Avant Filtre: 146
Après Filtre: 45


# Build Buoys_datas Dict

In [5]:
# Dictionnaire pour stocker les DataFrames, clé : ID de la bouée, valeur : DataFrame
buoy_datas = {}
buoy_list = []

# Parcours de chaque bouée dans stations_df
for index, row in stations_df.iterrows():
    buoy_id = row['Station']
    metadata = get_station_metadata(buoy_id)

    # ✅ Récupérer les données sous forme de dictionnaire
    buoy_info = parse_buoy_json(metadata)

    # ✅ Stocker directement les données dans buoy_datas
    buoy_datas[buoy_id] = buoy_info
    buoy_list.append(buoy_id)

# Affichage du nombre de bouées réussies et échouées
print(f"Nombre de bouées traitées : {len(buoy_datas)}\n")

# Afficher le contenu de buoy_datas

first_key =next(iter(buoy_datas))
first_key
buoy_datas[first_key]


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41044
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : south bermuda
🆔 Station ID : 41049
✅ Coordonnées extraites : Latitude = 27.50N, Longitude = 62.27W
🌊 Water Depth : 5480 m
🌡️ Sea Temp Depth : 1.5
🌬️ Barometer Elevation (m): 2.7
💨 An

{'station_zone': 'grays reef',
 'lat_buoy': '31.40N',
 'lon_buoy': '80.87W',
 'Water_depth': '16 m',
 'sea_temp_depth': '2',
 'Barometer_elevation': '2.4',
 'Anemometer_height': '3.8',
 'Air_temp_height': '3.4',
 'url': 'https://www.ndbc.noaa.gov/station_page.php?station=41008'}

# Collecte de données marines et météos

In [6]:
# 🚀 Démarrage du processus
print("\n🚀 Démarrage du processus de collecte des données...\n")

# Initialisation des compteurs
marine_data_collected_successfully = marine_data_collected_failed = 0
meteo_data_collected_successfully = meteo_data_collected_failed = 0

success = False
total_stations = stations_df.shape[0]
count = 0

# 🔄 Parcours des bouées / stations
for idx, row in stations_df.iterrows():
    buoy_id = row["Station"]

    ######### 🌊 MARINE DATA #########
    try:
        df_marine = NDBC.realtime_observations(buoy_id)
        if df_marine is None or df_marine.empty:
            marine_data_collected_failed += 1
            continue

        marine_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Ajout des métadonnées
    try:
        buoy_info = buoy_datas.get(buoy_id, {})
        Lat, Lon = buoy_info.get('lat_buoy'), buoy_info.get('lon_buoy')
        if Lat is None or Lon is None:
            raise ValueError(f"Données manquantes pour {buoy_id}")

        df_marine['Lat'] = Lat
        df_marine['Lon'] = Lon
        df_marine['Water_depth'] = buoy_info.get('Water_depth', None)
        df_marine.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_marine.columns]
        df_marine['Datetime'] = df_marine['Datetime'].dt.tz_localize(None)

        buoy_datas[buoy_id]["Marine"] = df_marine

        station_zone = safe_get(parse_buoy_json(get_station_metadata(buoy_id)), "station_zone")
        Bronze_Marine_table_Name = f"br_{buoy_id}_marine_{station_zone}".replace('.', '_').replace('-', '_').replace(' ', '_').lower()

    except Exception as e:
        print(f"⚠️ Erreur métadonnées marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    ######### ⛅ METEO DATA #########
    try:
        df_meteo = meteo_api_request([Lat, Lon])
        if df_meteo is None or df_meteo.empty:
            meteo_data_collected_failed += 1
            continue
        
        rename_columns(df_meteo, {'date':'Datetime'})
        df_meteo.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_meteo.columns]
        df_meteo['Datetime'] = df_meteo['Datetime'].dt.tz_localize(None)
    
        buoy_datas[buoy_id]["Meteo"] = df_meteo
        meteo_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte météo {buoy_id}: {e}")
        meteo_data_collected_failed += 1
        continue

# Retirer les bouées avec des DataFrames vides ou None
buoy_datas = {buoy_id: data for buoy_id, data in buoy_datas.items() 
              if "Marine" in data and data["Marine"] is not None and not data["Marine"].empty
              and "Meteo" in data and data["Meteo"] is not None and not data["Meteo"].empty}

# 🔚 Résumé final

print("\n📝 Résumé final :")
print(f"🌊 Marine - Collecte ✅ {marine_data_collected_successfully} ❌ {marine_data_collected_failed}")
print(f"⛅ Météo - Collecte ✅ {meteo_data_collected_successfully} ❌ {meteo_data_collected_failed}")

# Afficher la longueur du dictionnaire (nombre de bouées avec des données valides)
print(f"\n📊 Nombre de bouées avec des données valides : {len(buoy_datas)}")


🚀 Démarrage du processus de collecte des données...


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

📊 station_zone : grays reef
🔄 Colonne 'date' renommée en 'Datetime'
✅ Colonnes renommées : {'date': 'Datetime'}

🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41044
✅ Parsing terminé !

📊 station_zone : ne st martin
🔄 Colonne 'date' renommée en 'Datetime'
✅ Co

# Data Enrichment with MetaDatas

In [7]:
list_not_include = ['lon_buoy', "lat_buoy", "url"]
for buoy_id, value in buoy_datas.items():
    print(f"\n🔍 Traitement de la Station ID: {buoy_id}")

    marine_df = buoy_datas[buoy_id]["Marine"]
    meteo_df = buoy_datas[buoy_id]["Meteo"]

    try:
        # Récupérer les métadonnées de la station
        buoy_metadata = get_station_metadata(buoy_id)
        parsed_data = parse_buoy_json(buoy_metadata)

        # Mise à jour du dictionnaire avec les métadonnées
        data = buoy_datas[buoy_id]
        data.update(parsed_data)
        
        # Ajouter les métadonnées comme nouvelles colonnes dans marine_df
        if marine_df is not None:
            marine_df["Station ID"] = str(buoy_id)
            for key, value in parsed_data.items():
                # Vérifier si la clé n'est pas dans la liste des exclusions
                if key not in list_not_include:
                    marine_df[key] = value
                    print(f"✅ Colonne '{key}' ajoutée au DataFrame de la station {buoy_id}")

    except Exception as e:
        print(f"❌ Erreur pour la station {buoy_id}: {e}")

# Vérification de l'ajout des colonnes en prenant un id au hasard
station_id = random.choice(list(buoy_datas.keys()))
marine_df = buoy_datas[station_id]["Marine"]

if marine_df is not None:
    print("\nColonnes ajoutées au DataFrame de la station", station_id)
    print(marine_df.columns)


🔍 Traitement de la Station ID: 41008

🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

✅ Colonne 'station_zone' ajoutée au DataFrame de la station 41008
✅ Colonne 'Water_depth' ajoutée au DataFrame de la station 41008
✅ Colonne 'sea_temp_depth' ajoutée au DataFrame de la station 41008
✅ Colonne 'Barometer_elevation' ajoutée au DataFrame de la station 41008
✅ Colonne 'Anemometer_height' ajoutée au DataFrame de la station 41008
✅ Colonne 'Air_temp_height' ajoutée au DataFrame de la station 41008

🔍 Traitement de la Station ID: 41044

🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58

In [8]:
display_buoys_missing_df_counts(buoy_datas)


🌊 Nombre de bouées sans données 'Marine' : 0/43

☁️ Nombre de bouées sans données 'Meteo' : 0/43


In [9]:
display(df_marine.columns)
display(df_meteo.columns)

Index(['wind_direction', 'wind_speed', 'wind_gust', 'wave_height',
       'dominant_wave_period', 'average_wave_period',
       'dominant_wave_direction', 'pressure', 'air_temperature',
       'water_temperature', 'dewpoint', 'visibility', '3hr_pressure_tendency',
       'water_level_above_mean', 'Datetime', 'Lat', 'Lon', 'Water_depth',
       'Station ID', 'station_zone', 'sea_temp_depth', 'Barometer_elevation',
       'Anemometer_height', 'Air_temp_height'],
      dtype='object')

Index(['Datetime', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'precipitation', 'rain', 'showers', 'pressure_msl', 'surface_pressure',
       'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
       'visibility', 'wind_speed_10m', 'soil_temperature_0cm',
       'soil_moisture_0_to_1cm', 'is_day'],
      dtype='object')

# Handle Null Values

In [10]:
important_columns_oceanography = [
    'wind_direction',             
    'wind_speed',                 
    'wave_height',                   
    'pressure',                   
    'air_temperature',            
    'water_temperature',          
    'Datetime',
    'Lat',
    'Lon'                 
]

important_columns_meteorology = [
    'temperature_2m',             
    'relative_humidity_2m',       
    'dew_point_2m',               
    'precipitation',              
    'pressure_msl',               
    'cloud_cover',                
    'wind_speed_10m',             
    'Datetime'
]

stations_depart = len(buoy_datas)
ignored_buoys = {}  # Dictionary to track ignored buoys and their reasons

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Nettoyage des données pour la station {station_id}")

    marine_df = data.get("Marine")
    meteo_df = data.get("Meteo")

    if marine_df is None or meteo_df is None:
        ignored_buoys[station_id] = "Marine DataFrame ou Meteo DataFrame manquant(e)"
        print(f"⚠️ Station {station_id} ignorée: Marine DataFrame ou Meteo DataFrame manquant(e)")
        continue

    try:
        # Nettoyage des DataFrames
        cleaned_marine_df = handle_null_values(marine_df)
        cleaned_meteo_df = handle_null_values(meteo_df)
        # Vérification des colonnes importantes après nettoyage
        marine_columns_ok = all(col in cleaned_marine_df.columns for col in important_columns_oceanography)
        meteo_columns_ok = all(col in cleaned_meteo_df.columns for col in important_columns_meteorology)

        # Track which columns are missing
        missing_marine_columns = [col for col in important_columns_oceanography if col not in cleaned_marine_df.columns]
        missing_meteo_columns = [col for col in important_columns_meteorology if col not in cleaned_meteo_df.columns]

        if missing_marine_columns or missing_meteo_columns:
            ignored_buoys[station_id] = f"Colonnes manquantes: Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}"
            print(f"⚠️ Station {station_id} ignorée: Colonnes manquantes - Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}")
            continue

        # Ajouter le DataFrame nettoyé au dictionnaire des résultats
        buoy_datas[station_id]['Cleaned Marine'] = cleaned_marine_df
        buoy_datas[station_id]['Cleaned Meteo'] = cleaned_meteo_df
        print(f"✅ Nettoyage réussi pour la station {station_id} ({cleaned_marine_df.shape[0]} lignes)")

    except Exception as e:
        ignored_buoys[station_id] = f"Erreur lors du nettoyage: {e}"
        print(f"❌ Erreur lors du nettoyage pour {station_id}: {e}")

# 🔥 Suppression des stations ignorées du dictionnaire principal
for station_id in ignored_buoys:
    buoy_datas.pop(station_id, None)

len_cleaned_data = len([data for data in buoy_datas.values() if 'Cleaned Marine' in data and 'Cleaned Meteo' in data])

# Résumé final du nettoyage
print("\n📊 RÉSUMÉ DU NETTOYAGE:")
print(f"📌 Stations au départ : {stations_depart}")
print(f"✅ Stations nettoyées : {len_cleaned_data}")
print(f"🏁 Stations restantes après filtrage :")

for station_id, reason in ignored_buoys.items():
    print(f"🛑 Station {station_id} ignorée: {reason}")

print(f"\n🧹 Clés restantes dans buoy_datas après purge : {len(buoy_datas)} (attendu : {len_cleaned_data})")


🔄 Nettoyage des données pour la station 41008

Tag: orange - Nombre de lignes: 6541
Colonne 'wind_direction' Imputée par la médiane (16.82% de valeurs manquantes)
Colonne 'wind_speed' Imputée par la médiane (16.59% de valeurs manquantes)
Colonne 'wind_gust' Imputée par la médiane (16.59% de valeurs manquantes)
Colonne 'wave_height' Imputée par la médiane (49.01% de valeurs manquantes)
Colonne 'dominant_wave_period' Supprimée (plus de 55% de valeurs manquantes)
Colonne 'average_wave_period' Imputée par la médiane (49.03% de valeurs manquantes)
Colonne 'dominant_wave_direction' Imputée par la médiane (49.06% de valeurs manquantes)
Colonne 'pressure' Imputée par la médiane (0.17% de valeurs manquantes)
Colonne 'air_temperature' Imputée par la médiane (16.59% de valeurs manquantes)
Colonne 'water_temperature' Imputée par la médiane (12.7% de valeurs manquantes)
Colonne 'dewpoint' Imputée par la médiane (16.94% de valeurs manquantes)
Colonne 'visibility' Supprimée (plus de 55% de valeurs m

In [11]:
display_buoys_missing_df_counts(buoy_datas, prefix="Cleaned")


🌊 Nombre de bouées sans données 'Cleaned Marine' : 0/25

☁️ Nombre de bouées sans données 'Cleaned Meteo' : 0/25


In [12]:
display_row_values(df_meteo)

Index  |  Datetime             |  temperature_2m  |  relative_humidity_2m  |  dew_point_2m  |  precipitation  |  rain  |  showers  |  pressure_msl  |  surface_pressure  |  cloud_cover  |  cloud_cover_low  |  cloud_cover_mid  |  cloud_cover_high  |  visibility  |  wind_speed_10m  |  soil_temperature_0cm  |  soil_moisture_0_to_1cm  |  is_day
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0      |  2025-01-14 00:00:00  |  1.76            |  73.0                  |  -2.23         |  0.0            |  0.0   |  0.0      |  1015.6        |  990.675           |  90.5         |  25.0             |  0.0              |  0.0               |  22900.0     |  18.310001       |  3.99                  |  0.32                    | 

# Fusionner les df_meteo et df_marine sur 'Datetime'

In [13]:
# Fusion des DataFrames nettoyés
print("\n🔗 FUSION DES DONNÉES MARINE + METEO PAR STATION")

merged_success_count = 0  # Compteur de fusions réussies
total_merged_rows = 0     # Total de lignes fusionnées

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Fusion des données pour la station {station_id}")

    cleaned_marine_df = data.get("Cleaned Marine")
    cleaned_meteo_df = data.get("Cleaned Meteo")

    if cleaned_marine_df is None or cleaned_meteo_df is None:
        continue

    try:
        merged_df = pd.merge(cleaned_marine_df, cleaned_meteo_df, on="Datetime", how="inner")

        if merged_df.empty:
            print(f"⚠️ Station {station_id} fusionnée, mais résultat vide après inner merge sur 'Datetime'")
        else:
            buoy_datas[station_id]["Merged"] = merged_df
            merged_success_count += 1
            total_merged_rows += len(merged_df)
            print(f"✅ Fusion réussie pour la station {station_id} ({merged_df.shape[0]} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la fusion pour {station_id}: {e}")

# Résumé des fusions
print(f"\n📦 Fusions réussies : {merged_success_count}/{len_cleaned_data} stations")
print(f"📊 Total de lignes fusionnées : {total_merged_rows}")


🔗 FUSION DES DONNÉES MARINE + METEO PAR STATION

🔄 Fusion des données pour la station 41008
✅ Fusion réussie pour la station 41008 (1097 lignes)

🔄 Fusion des données pour la station 41044
✅ Fusion réussie pour la station 41044 (1089 lignes)

🔄 Fusion des données pour la station 41049
✅ Fusion réussie pour la station 41049 (1097 lignes)

🔄 Fusion des données pour la station 42001
✅ Fusion réussie pour la station 42001 (828 lignes)

🔄 Fusion des données pour la station 42002
✅ Fusion réussie pour la station 42002 (852 lignes)

🔄 Fusion des données pour la station 42020
✅ Fusion réussie pour la station 42020 (1097 lignes)

🔄 Fusion des données pour la station 42036
✅ Fusion réussie pour la station 42036 (1088 lignes)

🔄 Fusion des données pour la station 42056
✅ Fusion réussie pour la station 42056 (1097 lignes)

🔄 Fusion des données pour la station 42058
✅ Fusion réussie pour la station 42058 (1094 lignes)

🔄 Fusion des données pour la station 44007
✅ Fusion réussie pour la station 440

# Concaténation des DataFrames fusionnés

In [14]:
# Concaténation des DataFrames fusionnés
print("\n🧬 CONCATÉNATION DES DONNÉES FUSIONNÉES EN UN SEUL DATAFRAME")

final_merged_df_list = []
concat_success_count = 0
concat_total_rows = 0

for station_id, data in buoy_datas.items():
    merged_df = data.get("Merged")

    if merged_df is None:
        print(f"⚠️ Station {station_id} ignorée pour concaténation: Données fusionnées manquantes")
        continue

    try:
        final_merged_df_list.append(merged_df)
        concat_success_count += 1
        concat_total_rows += len(merged_df)
        print(f"✅ Concaténation réussie pour la station {station_id} ({len(merged_df)} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la concaténation pour {station_id}: {e}")

# Création du DataFrame final unique
try:
    df_final = pd.concat(final_merged_df_list, ignore_index=True)
    print(f"\n🧾 DataFrame final créé avec succès ({df_final.shape[0]} lignes, {df_final.shape[1]} colonnes)")
except Exception as e:
    print(f"\n❌ Erreur lors de la création du DataFrame final: {e}")
    df_final = None

# Résumé
print(f"\n📦 Concaténations réussies : {concat_success_count}/{merged_success_count}")
print(f"📊 Total de lignes dans le DataFrame final : {concat_total_rows}")


🧬 CONCATÉNATION DES DONNÉES FUSIONNÉES EN UN SEUL DATAFRAME
✅ Concaténation réussie pour la station 41008 (1097 lignes)
✅ Concaténation réussie pour la station 41044 (1089 lignes)
✅ Concaténation réussie pour la station 41049 (1097 lignes)
✅ Concaténation réussie pour la station 42001 (828 lignes)
✅ Concaténation réussie pour la station 42002 (852 lignes)
✅ Concaténation réussie pour la station 42020 (1097 lignes)
✅ Concaténation réussie pour la station 42036 (1088 lignes)
✅ Concaténation réussie pour la station 42056 (1097 lignes)
✅ Concaténation réussie pour la station 42058 (1094 lignes)
✅ Concaténation réussie pour la station 44007 (1096 lignes)
✅ Concaténation réussie pour la station 44020 (1096 lignes)
✅ Concaténation réussie pour la station 44065 (1098 lignes)
✅ Concaténation réussie pour la station 46006 (1094 lignes)
✅ Concaténation réussie pour la station 46014 (1097 lignes)
✅ Concaténation réussie pour la station 46025 (1096 lignes)
✅ Concaténation réussie pour la station 4

In [15]:
display_row_values(df_final)

Index  |  wind_direction  |  wind_speed  |  wind_gust  |  wave_height  |  average_wave_period  |  dominant_wave_direction  |  pressure  |  air_temperature  |  water_temperature  |  dewpoint  |  Datetime             |  Lat     |  Lon      |  Water_depth  |  Station ID  |  station_zone                              |  sea_temp_depth  |  Barometer_elevation  |  Anemometer_height  |  Air_temp_height  |  temperature_2m  |  relative_humidity_2m  |  dew_point_2m  |  precipitation  |  rain  |  showers  |  pressure_msl  |  surface_pressure  |  cloud_cover  |  cloud_cover_low  |  cloud_cover_mid  |  cloud_cover_high  |  visibility  |  wind_speed_10m  |  soil_temperature_0cm  |  soil_moisture_0_to_1cm  |  is_day
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Hour Filtering

In [16]:
try:
    df_final['Datetime'] = pd.to_datetime(df_final['Datetime'])

    df_final = df_final[df_final['Datetime'].dt.minute == 0]
    # Affichage pour vérifier le résultat
    print(f"🚀 DataFrame filtrée pour ne garder que les lignes à l'heure pile: {df_final.shape[0]} lignes")

except Exception as e:
    print(f"�� Erreur lors du filtrage des lignes à l'heure pile: {e}")

🚀 DataFrame filtrée pour ne garder que les lignes à l'heure pile: 26879 lignes


In [17]:
final_datetime = pd.DataFrame(df_final[['Datetime']], dtype='datetime64[ns]')
final_datetime

,Datetime
0,2025-04-16 17:00:00
1,2025-04-16 16:00:00
2,2025-04-16 15:00:00
3,2025-04-16 14:00:00
4,2025-04-16 13:00:00
...,...
26874,2025-03-02 04:00:00
26875,2025-03-02 03:00:00
26876,2025-03-02 02:00:00
26877,2025-03-02 01:00:00


In [18]:
df_final = handle_null_values(df_final)
display_row_values(df_final)


Tag: yellow - Nombre de lignes: 26879
Colonne 'wind_direction' non modifiée (0% de valeurs manquantes)
Colonne 'wind_speed' non modifiée (0% de valeurs manquantes)
Colonne 'wind_gust' non modifiée (0% de valeurs manquantes)
Colonne 'wave_height' non modifiée (0% de valeurs manquantes)
Colonne 'average_wave_period' non modifiée (0% de valeurs manquantes)
Colonne 'dominant_wave_direction' Imputée par la médiane (4.08% de valeurs manquantes)
Colonne 'pressure' non modifiée (0% de valeurs manquantes)
Colonne 'air_temperature' non modifiée (0% de valeurs manquantes)
Colonne 'water_temperature' non modifiée (0% de valeurs manquantes)
Colonne 'dewpoint' non modifiée (0% de valeurs manquantes)
Colonne 'Datetime' non modifiée (0% de valeurs manquantes)
Colonne 'Lat' non modifiée (0% de valeurs manquantes)
Colonne 'Lon' non modifiée (0% de valeurs manquantes)
Colonne 'Water_depth' non modifiée (0% de valeurs manquantes)
Colonne 'Station ID' non modifiée (0% de valeurs manquantes)
Colonne 'stati

In [19]:
# get station id distinct values
station_ids = df_final['Station ID'].unique()
print(station_ids)

['41008' '41044' '41049' '42001' '42002' '42020' '42036' '42056' '42058'
 '44007' '44020' '44065' '46006' '46014' '46025' '46027' '46072' '46078'
 '46084' '46086' '46087' '46088' '51000' '51001' '51002']


In [20]:
df_final.columns = [col.strip() for col in df_final.columns]

# Renaming and deleting useless columns

In [21]:
# Dictionnaire de renommage des colonnes
col_to_rename = {'temperature_2m': 'T°(C°)', 
                 'relative_humidity_2m': 'Relative Humidity (%)',
                 'dew_point_2m': 'Dew Point (°C)', 
                 'precipitation': 'Precipitations (mm)',  
                 'pressure_msl':'Sea Level Pressure (hPa)', 
                 'cloud_cover_low':'Low Clouds (%)',
                 'cloud_cover_mid' : 'Middle Clouds (%)', 
                 'cloud_cover_high' : 'High Clouds (%)', 
                 'visibility' : 'Visibility (km)', 
                 'wind_direction': 'Wind Direction (°)',
                 'wind_speed': 'Wind Speed (km/h)', 
                 'wind_gust': 'Wind Gusts (km/h)',
                 'wind_speed_10m':'Wind Speed (10m)', 
                 'surface_pressure': 'Surface Pressure',
                 'wave_height': 'Wave Height (m)', 
                 'average_wave_period': 'Average Wave Period (s)',
                 'dominant_wave_direction': 'Dominant Wave Direction (°)', 
                 'pressure': 'Pressure (hPa)',
                 'air_temperature': 'Air T°', 
                 'water_temperature': 'Water T°', 
                 'Water_depth': 'Water Depth (m)', 
                 "Air_temp_height": "Air T° Height", 
                 "Anemometer_height": "Anemometer Height (m)", 
                 "station_zone": "Station Zone",
                 "Barometer_elevation": "Barometer Elevation", 
                 "sea_temp_depth" : "Sea Temperature Depth (m)",
                 "cloud_cover": "Cloud Cover (%)"
                 }


# Liste des colonnes à supprimer
cols_to_delete = ['soil_temperature_0cm', 'lat_buoy','lon_buoy', 'rain', 
                  'showers', 'is_day', 'soil_moisture_0_to_1cm']
	
# Renommer les colonnes d'abord
df_final = rename_columns(df_final, col_to_rename)
# Ensuite, supprimer les colonnes non désirées
df_final = drop_columns_if_exist(df_final, cols_to_delete)
try:
    if df_final['Visibility (km)'].mean() > 1000:
        df_final['Visibility (km)'] = df_final['Visibility (km)'] / 1000
        print("Conversion de la visibilité de mètres à kilomètres")
    df_final["T°(C°)"] = round(df_final["T°(C°)"], 2)
    df_final["Wind Speed (10m)"] = round(df_final["Wind Speed (10m)"], 2)
except Exception as e:
    print(f"�� Erreur lors du traitement des colonnes :\n {e}")

# Afficher les résultats
print("\nColonnes après renommage et suppression :")
display_row_values(df_final)

🔄 Colonne 'temperature_2m' renommée en 'T°(C°)'
🔄 Colonne 'relative_humidity_2m' renommée en 'Relative Humidity (%)'
🔄 Colonne 'dew_point_2m' renommée en 'Dew Point (°C)'
🔄 Colonne 'precipitation' renommée en 'Precipitations (mm)'
🔄 Colonne 'pressure_msl' renommée en 'Sea Level Pressure (hPa)'
🔄 Colonne 'cloud_cover_low' renommée en 'Low Clouds (%)'
🔄 Colonne 'cloud_cover_mid' renommée en 'Middle Clouds (%)'
🔄 Colonne 'cloud_cover_high' renommée en 'High Clouds (%)'
🔄 Colonne 'visibility' renommée en 'Visibility (km)'
🔄 Colonne 'wind_direction' renommée en 'Wind Direction (°)'
🔄 Colonne 'wind_speed' renommée en 'Wind Speed (km/h)'
🔄 Colonne 'wind_gust' renommée en 'Wind Gusts (km/h)'
🔄 Colonne 'wind_speed_10m' renommée en 'Wind Speed (10m)'
🔄 Colonne 'surface_pressure' renommée en 'Surface Pressure'
🔄 Colonne 'wave_height' renommée en 'Wave Height (m)'
🔄 Colonne 'average_wave_period' renommée en 'Average Wave Period (s)'
🔄 Colonne 'dominant_wave_direction' renommée en 'Dominant Wave Di

In [22]:
try:
    display_row_values(df_final, columns=["Visibility (km)", "T°(C°)"])
#    df_final["Visibility (km)"] = df_final["Visibility (km)"].map(lambda x: x / 1000)
    
    df_final["T°(C°)"] = df_final["T°(C°)"].round(2)
    df_final[['Lat', 'Lon']] = df_final.apply(
        lambda row: pd.Series(convert_coordinates(row['Lat'], row['Lon'])),
        axis=1
    )
except Exception as e:
    print(f"Erreur : {e}")
display_row_values(df_final)

Index  |  Visibility (km)  |  T°(C°)
----------------------------------
0      |  37.8             |  17.3  
1      |  32.0             |  16.7  
2      |  31.6             |  17.35 
3      |  33.7             |  17.4  
4      |  33.2             |  17.75 
5      |  37.4             |  18.15 
6      |  35.9             |  18.5  
7      |  40.1             |  18.55 
8      |  39.8             |  18.85 
9      |  37.4             |  18.55 
Index  |  Wind Direction (°)  |  Wind Speed (km/h)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Pressure (hPa)  |  Air T°  |  Water T°  |  dewpoint  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Anemometer Height (m)  |  Air T° Height  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Surface Pres

In [23]:
#  T°(C°) = (Air T° + T°(C°))/2 et virer Air T°
try:
    df_final['T°(C°)'] = (df_final['Air T°'] + df_final['T°(C°)']) / 2
    df_final['T°(C°)'] = df_final['T°(C°)'].round(2)
    df_final.drop(columns=['Air T°'], inplace=True)
    df_final['Sea Level Pressure (hPa)'] = round((df_final['Sea Level Pressure (hPa)'] + df_final['Surface Pressure']) / 2, 2)
    df_final.drop(columns=['Surface Pressure'], inplace=True)
    df_final['Year'] = df_final['Datetime'].dt.year
    df_final['Month'] = df_final['Datetime'].dt.month_name()
    df_final['DayOfWeek'] = df_final['Datetime'].dt.day_name()
    df_final['DayPeriod'] = df_final['Datetime'].apply(
    lambda x: 'Morning' if 6 <= x.hour < 12 else
              'Afternoon' if 12 <= x.hour < 18 else
              'Evening' if 18 <= x.hour < 22 else
              'Night'
)
    df_final.drop(columns='Hour', inplace=True)
except Exception as e:
    print(f"Erreur :\n {e}")

# virer le m dans Water Depth avec regex lambda et passer la colonne en float
df_final['Water Depth (m)'] = df_final['Water Depth (m)'].apply(lambda x: re.sub(r'\D', '', str(x)).strip())
df_final['Water Depth (m)'] = df_final['Water Depth (m)'].astype(float)
display_row_values(df_final)

Erreur :
 "['Hour'] not found in axis"
Index  |  Wind Direction (°)  |  Wind Speed (km/h)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Pressure (hPa)  |  Water T°  |  dewpoint  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Anemometer Height (m)  |  Air T° Height  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Cloud Cover (%)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Year  |  Month  |  DayOfWeek  |  DayPeriod
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Check truth about Wind Speed Using another API call

In [24]:
df_42058 = df_final[df_final['Station ID'] == "42058"]
df_42058.columns

Index(['Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPa)', 'Water T°', 'dewpoint',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Anemometer Height (m)', 'Air T° Height', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
       'Sea Level Pressure (hPa)', 'Cloud Cover (%)', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)',
       'Wind Speed (10m)', 'Year', 'Month', 'DayOfWeek', 'DayPeriod'],
      dtype='object')

# Requête à l'API Visual Crossing pour les données de vérification (1 / 24h)

In [25]:
# ---- Chargement de la clé API ----
vc_api_key_path = r"c:\Credentials\visual_crossing_weather_api.json"
with open(vc_api_key_path, 'r') as file:
    content = json.load(file)
    vc_api_key = content["api_key"]

# ---- Extraire les coordonnées depuis la première ligne du DataFrame ----
lat_42058, lon_42058 = None, None

if not df_42058.empty:
    first_row = df_42058.iloc[0]
    lat_42058, lon_42058 = first_row["Lat"], first_row["Lon"]

# ---- Définir les dates pour la requête ----
today = datetime.now().strftime("%Y-%m-%d")
last_month = (datetime.now() - timedelta(days=31)).strftime("%Y-%m-%d")

# ---- Créer le dossier de cache si nécessaire ----
cache_dir = "api_call_files"
os.makedirs(cache_dir, exist_ok=True)

# ---- Définir le fichier cache selon la position ----
cache_file = os.path.join(cache_dir, f"vc_meteo_{lat_42058}_{lon_42058}.csv")

# ---- Vérifier si un cache récent existe (moins de 24h) ----
use_cache = False
if os.path.exists(cache_file):
    last_modified = datetime.fromtimestamp(os.path.getmtime(cache_file))
    if datetime.now() - last_modified < timedelta(hours=24):
        print(f"📦 Cache détecté ({cache_file}), modifié le {last_modified.strftime('%Y-%m-%d %H:%M:%S')}")
        vc_meteo_df = pd.read_csv(cache_file)
        print("✅ Données météo rechargées depuis le cache.")
        use_cache = True
    else:
        print(f"⚠️ Cache trouvé mais périmé (plus de 24h) → nouvelle requête API.")

# ---- Appel API si pas de cache valide ----
if not use_cache and lat_42058 is not None and lon_42058 is not None:
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat_42058},{lon_42058}/{last_month}/{today}?unitGroup=metric&key={vc_api_key}&contentType=json"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            vc_meteo_data = response.json()
            print("🌍 Données météo récupérées depuis l'API Visual Crossing.")

            # ---- Extraire les données journalières et sauvegarder ----
            if "days" in vc_meteo_data:
                vc_meteo_df = pd.json_normalize(vc_meteo_data["days"])
                vc_meteo_df.to_csv(cache_file, index=False)
                print(f"💾 Données sauvegardées dans le cache : {cache_file}")
            else:
                print("⚠️ Le champ 'days' est absent de la réponse API.")
        else:
            print(f"❌ Échec de l’appel API — code de statut : {response.status_code}")
    except Exception as e:
        print(f"❌ Exception levée lors de la requête API : {e}")
        
# 📁 Charger les données du CSV Visual Crossing
vc_csv_path = f"api_call_files/vc_meteo_{lat_42058}_{lon_42058}.csv"
df_vc_meteo = pd.read_csv(vc_csv_path)
df_vc_meteo.head()

📦 Cache détecté (api_call_files\vc_meteo_14.51_-75.15.csv), modifié le 2025-04-16 10:09:52
✅ Données météo rechargées depuis le cache.


,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,hours
0,2025-03-16,1742101200,27.5,27.0,27.3,30.6,29.3,30.0,22.9,77.0,...,1742123237,18:11:11,1742166671,0.58,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,['42058_maritime'],obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
1,2025-03-17,1742187600,27.5,26.5,27.3,30.8,26.5,30.1,23.4,79.6,...,1742209595,18:11:18,1742253078,0.61,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['42058_maritime', 'remote']",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
2,2025-03-18,1742274000,27.4,26.8,27.1,30.1,28.7,29.4,22.4,75.7,...,1742295953,18:11:25,1742339485,0.65,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['42058_maritime', 'remote']",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
3,2025-03-19,1742360400,27.4,26.7,27.2,30.3,26.7,29.5,22.3,74.4,...,1742382311,18:11:32,1742425892,0.68,Clear,Clear conditions throughout the day.,clear-day,"['42058_maritime', 'remote']",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
4,2025-03-20,1742446800,27.5,26.5,27.3,30.9,26.5,29.9,22.9,77.4,...,1742468669,18:11:39,1742512299,0.71,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,"['42058_maritime', 'remote']",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."


# Nettoyage du DataFrame retourné

In [26]:
# 🧼 Nettoyage et transformation
  # Permet de convertir les strings représentant des listes en objets Python
all_hours = []

# Parcours de chaque ligne du fichier météo
for i, row in df_vc_meteo.iterrows():
    try:
        # Convertir la colonne "hours" en liste de dictionnaires
        hours_list = ast.literal_eval(row['hours'])

        # Pour chaque heure dans la journée, on ajoute une entrée au tableau final
        for hour_data in hours_list:
            hour_data['Date'] = row['datetime']  # On ajoute aussi la date du jour correspondant
            all_hours.append(hour_data)

    except Exception as e:
        print(f"Erreur parsing ligne {i}: {e}")  # En cas d'erreur de parsing (souvent problème de format)

# ✅ On transforme la liste aplanie en DataFrame
df_vc_flat = pd.DataFrame(all_hours)

# 🕒 Conversion de l'heure/temps
df_vc_flat["Datetime"] = pd.to_datetime(df_vc_flat["datetimeEpoch"], unit="s").dt.strftime("%Y-%m-%d-%H")

# 🗓️ Filtrer sur les 30 derniers jours
today = datetime.now()
thirty_days_ago = today - timedelta(days=30)

today_str = today.strftime("%Y-%m-%d")
thirty_days_ago_str = thirty_days_ago.strftime("%Y-%m-%d")

# On filtre le DataFrame pour ne garder que les lignes entre ces deux dates
df_vc_last_month = df_vc_flat[
    (df_vc_flat['Date'] >= thirty_days_ago_str) & 
    (df_vc_flat['Date'] <= today_str)]

display_row_values(df_vc_last_month)

Index  |  datetime  |  datetimeEpoch  |  temp  |  feelslike  |  humidity  |  dew   |  precip  |  precipprob  |  snow  |  snowdepth  |  preciptype  |  windgust  |  windspeed  |  winddir  |  pressure  |  visibility  |  cloudcover  |  solarradiation  |  solarenergy  |  uvindex  |  severerisk  |  conditions              |  icon                 |  stations            |  source  |  Date        |  Datetime     
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
24     |  00:00:00  |  1742187600     |  27.4  |  30.4       |  78.83     |  23.4  |  0.0     |  0.0         |  0.0   |  0.0        |  None        |  41.4      |  28.8       |  80.0     |  1014.5    |  24.1     

# Renommage des colonnes pour faciliter la comparaison

In [27]:
# Correct syntaxe pour filtrer les colonnes spécifiques
df_vc_last_month = df_vc_last_month[["Datetime", "temp", "humidity", "precip", "dew", "windgust", 
                                     "windspeed", "winddir", "pressure", "visibility"]]

for col in df_vc_last_month.columns:
    if not col == "Datetime":
        rename_columns(df_vc_last_month, {col: f"VC_{col}"})

🔄 Colonne 'temp' renommée en 'VC_temp'
✅ Colonnes renommées : {'temp': 'VC_temp'}
🔄 Colonne 'humidity' renommée en 'VC_humidity'
✅ Colonnes renommées : {'humidity': 'VC_humidity'}
🔄 Colonne 'precip' renommée en 'VC_precip'
✅ Colonnes renommées : {'precip': 'VC_precip'}
🔄 Colonne 'dew' renommée en 'VC_dew'
✅ Colonnes renommées : {'dew': 'VC_dew'}
🔄 Colonne 'windgust' renommée en 'VC_windgust'
✅ Colonnes renommées : {'windgust': 'VC_windgust'}
🔄 Colonne 'windspeed' renommée en 'VC_windspeed'
✅ Colonnes renommées : {'windspeed': 'VC_windspeed'}
🔄 Colonne 'winddir' renommée en 'VC_winddir'
✅ Colonnes renommées : {'winddir': 'VC_winddir'}
🔄 Colonne 'pressure' renommée en 'VC_pressure'
✅ Colonnes renommées : {'pressure': 'VC_pressure'}
🔄 Colonne 'visibility' renommée en 'VC_visibility'
✅ Colonnes renommées : {'visibility': 'VC_visibility'}


Enforce Datetime Format

In [28]:
# Convert 'Datetime' column to datetime type in df_vc_last_month
df_vc_last_month['Datetime'] = pd.to_datetime(df_vc_last_month['Datetime'], errors='coerce')

# Check if the Datetime column is correctly converted
print(df_42058['Datetime'].dtype)
print(df_vc_last_month['Datetime'].dtype)

datetime64[ns]
datetime64[ns]


Merge and Compare DataFrames

In [29]:
# Now perform the merge
df_compare = pd.merge(df_42058, df_vc_last_month, on="Datetime", how="inner")
df_compare.columns

Index(['Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPa)', 'Water T°', 'dewpoint',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Anemometer Height (m)', 'Air T° Height', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
       'Sea Level Pressure (hPa)', 'Cloud Cover (%)', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)',
       'Wind Speed (10m)', 'Year', 'Month', 'DayOfWeek', 'DayPeriod',
       'VC_temp', 'VC_humidity', 'VC_precip', 'VC_dew', 'VC_windgust',
       'VC_windspeed', 'VC_winddir', 'VC_pressure', 'VC_visibility'],
      dtype='object')

Comparaison Wind Speed

In [30]:
try:
    # Wind Speed Comparison
    df_windspeed_compare = df_compare[['Wind Speed (km/h)', 'Wind Speed (10m)','VC_windspeed']]
    #  Pressure Comparison
    df_pressure_compare = df_compare[['Sea Level Pressure (hPa)','Pressure (hPa)', 'VC_pressure']]
    # Dew Point Comparison
    df_dew_compare = df_compare[['dewpoint','Dew Point (°C)', 'VC_dew']]
except Exception as e:
    print(e)

df_windspeed_compare.head(10)

,Wind Speed (km/h),Wind Speed (10m),VC_windspeed
0,8.0,33.119999,29.2
1,9.0,33.860001,31.0
2,8.0,35.189999,29.9
3,9.0,35.889999,31.7
4,8.0,34.759998,32.8
5,9.0,34.040001,32.4
6,9.0,36.939999,32.8
7,9.0,37.380001,33.1
8,8.0,35.520000,33.5
9,8.0,35.570000,33.1


In [31]:
wind_col_to_delete = ['Wind Speed (km/h)', 'Anemometer Height (m)']
df_final = drop_columns_if_exist(df_final, wind_col_to_delete)

 Nombre initial de colonnes: 34
Colonne 'Wind Speed (km/h)' Supprimée
Colonne 'Anemometer Height (m)' Supprimée
 Nombre final de colonnes: 32


Comparaison Pressure

In [32]:
df_pressure_compare.head()

,Sea Level Pressure (hPa),Pressure (hPa),VC_pressure
0,1011.900024,1011.7,1013.0
1,1012.599976,1012.1,1013.0
2,1013.000000,1012.3,1013.0
3,1013.200012,1012.2,1013.0
4,1013.000000,1011.7,1013.0


In [33]:
df_final = drop_columns_if_exist(df_final, ['Pressure (hPa)'])

 Nombre initial de colonnes: 32
Colonne 'Pressure (hPa)' Supprimée
 Nombre final de colonnes: 31


In [34]:
df_dew_compare.head(20)

,dewpoint,Dew Point (°C),VC_dew
0,24.1,23.469999,23.7
1,24.0,23.420000,23.6
2,24.5,23.690001,23.9
3,24.4,23.690001,24.0
4,24.2,23.590000,24.0
5,24.2,23.910000,23.9
6,24.0,23.860001,23.8
7,24.4,23.750000,23.8
8,24.1,23.860001,23.8
9,24.5,23.910000,23.8


Code Custom GPT Wisdom of crowd pour le DewPoint

In [35]:
try:

    # Calcul des distances absolues entre chaque paire de mesures pour chaque ligne
    # Ces distances nous permettent de savoir quelle mesure est la plus proche des autres
    df_compare['dist_dewpoint_DewPoint'] = np.abs(df_compare['dewpoint'] - df_compare['Dew Point (°C)'])
    df_compare['dist_dewpoint_VC'] = np.abs(df_compare['dewpoint'] - df_compare['VC_dew'])
    df_compare['dist_DewPoint_VC'] = np.abs(df_compare['Dew Point (°C)'] - df_compare['VC_dew'])

    # Pour chaque ligne, on détermine quelle mesure est la plus proche des deux autres :
    # - Si 'dewpoint' est plus proche des autres mesures que 'Dew Point (°C)' et 'VC_dew', alors 'dewpoint' est marqué comme plus proche.
    df_compare['dewpoint_closer'] = (df_compare['dist_dewpoint_VC'] < df_compare['dist_dewpoint_DewPoint']) & (df_compare['dist_dewpoint_VC'] < df_compare['dist_DewPoint_VC'])

    # - Si 'Dew Point (°C)' est plus proche des autres mesures que 'dewpoint' et 'VC_dew', alors 'Dew Point (°C)' est marqué comme plus proche.
    df_compare['DewPoint_closer'] = (df_compare['dist_dewpoint_DewPoint'] < df_compare['dist_dewpoint_VC']) & (df_compare['dist_dewpoint_DewPoint'] < df_compare['dist_DewPoint_VC'])

    # - Si 'VC_dew' est plus proche des autres mesures que 'dewpoint' et 'Dew Point (°C)', alors 'VC_dew' est marqué comme plus proche.
    df_compare['VC_closer'] = (df_compare['dist_DewPoint_VC'] < df_compare['dist_dewpoint_VC']) & (df_compare['dist_DewPoint_VC'] < df_compare['dist_dewpoint_DewPoint'])

    # Calculer les probabilités que chaque mesure soit la plus proche des autres sur l'ensemble des lignes :
    # La probabilité est simplement la proportion de fois où une mesure a été plus proche des autres.
    prob_dewpoint_closer = df_compare['dewpoint_closer'].mean().round(3)
    prob_dewpoint_c_closer = df_compare['DewPoint_closer'].mean().round(3)
    prob_vc_closer = df_compare['VC_closer'].mean().round(3)

    # Afficher les résultats
    # Ces résultats indiquent la probabilité que chaque mesure soit la plus proche des autres sur toutes les lignes de données
    print(f"Probability that 'dewpoint' is closer to the truth: {prob_dewpoint_closer}")
    print(f"Probability that 'Dew Point (°C)' is closer to the truth: {prob_dewpoint_c_closer}")
    print(f"Probability that 'VC_dew' is closer to the truth: {prob_vc_closer}")
    
except Exception as e:
    print(e)

Probability that 'dewpoint' is closer to the truth: 0.807
Probability that 'Dew Point (°C)' is closer to the truth: 0.089
Probability that 'VC_dew' is closer to the truth: 0.096


In [36]:
try:
    df_final['Dew Point (°C)'] = df_final['dewpoint']
    df_final = drop_columns_if_exist(df_final, ['dewpoint'])
    
except Exception as e:
    print(e)
display(df_final.info())
display_row_values(df_final)

 Nombre initial de colonnes: 31
Colonne 'dewpoint' Supprimée
 Nombre final de colonnes: 30
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26879 entries, 0 to 26878
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Wind Direction (°)           26879 non-null  float64       
 1   Wind Gusts (km/h)            26879 non-null  float64       
 2   Wave Height (m)              26879 non-null  float64       
 3   Average Wave Period (s)      26879 non-null  float64       
 4   Dominant Wave Direction (°)  26879 non-null  float64       
 5   Water T°                     26879 non-null  float64       
 6   Datetime                     26879 non-null  datetime64[ns]
 7   Lat                          26879 non-null  float64       
 8   Lon                          26879 non-null  float64       
 9   Water Depth (m)              26879 non-null  float64       
 10  Station ID     

None

Index  |  Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T°  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Air T° Height  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Cloud Cover (%)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Year  |  Month  |  DayOfWeek  |  DayPeriod
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Rename Final Columns

In [37]:
try:
    df_final.rename(columns={'Air T° Height': 'Air T° Height (m)',
                             'Barometer Elevation': 'Barometer Elevation (m)',
                             'Water T°': 'Water T° (°C)'}, inplace=True)

    print("Column 'Air T° Height' renamed to 'Air T° Height (m)'")

    print("Column 'Barometer Elevation' renamed to 'Barometer Elevation (m)'")
    print("Column 'Water T°' renamed to 'Water T° (°C)'")
    df_final['T°(C°)'] = df_final['T°(C°)'].round(2)

    print("Column 'T°(C°)' rounded to 2 decimal places")
except Exception as e:
    print(e)
    
display_row_values(df_final)

Column 'Air T° Height' renamed to 'Air T° Height (m)'
Column 'Barometer Elevation' renamed to 'Barometer Elevation (m)'
Column 'Water T°' renamed to 'Water T° (°C)'
Column 'T°(C°)' rounded to 2 decimal places
Index  |  Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T° (°C)  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation (m)  |  Air T° Height (m)  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Cloud Cover (%)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Year  |  Month  |  DayOfWeek  |  DayPeriod
--------------------------------------------------------------------------------------------------------------------------------------------------------------

In [38]:
df_final.dtypes

Wind Direction (°)                    float64
Wind Gusts (km/h)                     float64
Wave Height (m)                       float64
Average Wave Period (s)               float64
Dominant Wave Direction (°)           float64
Water T° (°C)                         float64
Datetime                       datetime64[ns]
Lat                                   float64
Lon                                   float64
Water Depth (m)                       float64
Station ID                             object
Station Zone                           object
Sea Temperature Depth (m)              object
Barometer Elevation (m)                object
Air T° Height (m)                      object
T°(C°)                                float64
Relative Humidity (%)                 float32
Dew Point (°C)                        float64
Precipitations (mm)                   float32
Sea Level Pressure (hPa)              float32
Cloud Cover (%)                       float32
Low Clouds (%)                    

In [80]:
# Correction des types des colonnes
try:

    df_final['Barometer Elevation (m)'] = df_final['Barometer Elevation (m)'].astype(str)

    df_final['Air T° Height (m)'] = df_final['Air T° Height (m)'].astype(str)
    df_final['Sea Temperature Depth (m)'] = df_final['Sea Temperature Depth (m)'].astype(str)
    df_final['Lat'] = df_final['Lat'].astype(str)
    df_final['Lon'] = df_final['Lon'].astype(str)
    df_final['Year'] = df_final['Year'].astype(str)
    print(df_final.dtypes)
except Exception as e:
    print(e)

Lat                             object
Lon                             object
Wind Direction (°)             float64
Wind Gusts (km/h)              float64
Wave Height (m)                float64
Average Wave Period (s)        float64
Dominant Wave Direction (°)    float64
Water T° (°C)                  float64
Water Depth (m)                float64
Station ID                      object
Station Zone                    object
Sea Temperature Depth (m)       object
Barometer Elevation (m)         object
Air T° Height (m)               object
T°(C°)                         float64
Relative Humidity (%)          float32
Dew Point (°C)                 float64
Precipitations (mm)            float32
Sea Level Pressure (hPa)       float32
Cloud Cover (%)                float32
Low Clouds (%)                 float32
Middle Clouds (%)              float32
High Clouds (%)                float32
Visibility (km)                float32
Wind Speed (10m)               float32
Year                     

In [40]:
try:
    df_final = df_final[['Datetime', 'Lat', 'Lon'] + [col for col in df_final.columns if col not in ['Datetime', 'Lat', 'Lon']]]
    # placer la colonne Datetime en %Y-%m-%d %H
    
    df_final['Datetime'] = pd.to_datetime(df_final['Datetime'], errors='coerce')
    df_final = pd.to_datetime(df_final.index).dt.floor('min')
    df_final['Datetime'] = df_final['Datetime'].dt.strftime('%Y-%m-%d %H:%M')
    
    df_final.Datetime
except Exception as e:
    print(str(e))

'DatetimeIndex' object has no attribute 'dt'


In [41]:
try:
    df_final.set_index('Datetime', inplace=True)
except Exception as e:
    print(str(e))
    display_row_values(df_final)

In [42]:
display_row_values(df_final)

Datetime             |  Lat    |  Lon      |  Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T° (°C)  |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation (m)  |  Air T° Height (m)  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Cloud Cover (%)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Year  |  Month  |  DayOfWeek  |  DayPeriod
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [43]:
try:
    create_table_in_mysql(df=df_final, engine=engine, table_name=silver_table)
    insert_new_rows(df=df_final, engine=engine, table_name=silver_table, ref='index')
except Exception as e:
    print(str(e))

✅ Table 'silver table' créée avec succès via SQLAlchemy.
📋 Récupération de la table 'silver table' dans la base de données...
🔎 Vérification des valeurs uniques de l'index...
✅ 0 références existantes trouvées.
🚀 Insertion de 26879 nouvelles lignes...
(mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry '2025-04-16 17:00:00' for key 'silver table.PRIMARY'
[SQL: INSERT INTO `silver table` (`Datetime`, `Lat`, `Lon`, `Wind Direction (°)`, `Wind Gusts (km/h)`, `Wave Height (m)`, `Average Wave Period (s)`, `Dominant Wave Direction (°)`, `Water T° (°C)`, `Water Depth (m)`, `Station ID`, `Station Zone`, `Sea Temperature Depth (m)`, `Barometer Elevation (m)`, `Air T° Height (m)`, `T°(C°)`, `Relative Humidity (%)`, `Dew Point (°C)`, `Precipitations (mm)`, `Sea Level Pressure (hPa)`, `Cloud Cover (%)`, `Low Clouds (%)`, `Middle Clouds (%)`, `High Clouds (%)`, `Visibility (km)`, `Wind Speed (10m)`, `Year`, `Month`, `DayOfWeek`, `DayPeriod`) VALUES (%(Datetime)s, %(Lat)s, %(Lon)s,

In [44]:
# envoyer la df en csv 
df_final.to_csv('data_final.csv', index=False)

In [45]:
display_row_values(df_final)

Datetime             |  Lat    |  Lon      |  Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T° (°C)  |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation (m)  |  Air T° Height (m)  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Cloud Cover (%)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Year  |  Month  |  DayOfWeek  |  DayPeriod
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 

Création des DataFrames pour les tables

In [65]:
# Garder l'index sous forme de datetime pour les calculs
df_final_copy = df_final.copy()

df_final_copy.index = pd.to_datetime(df_final_copy.index).floor('min')

# Pour l'affichage sans secondes
df_final_copy['Datetime'] = df_final_copy.index.strftime('%Y-%m-%d %H:%M')

# Nettoyer Station ID
df_final_copy['Station ID'] = df_final_copy['Station ID'].astype(str).str.strip()

# Créer les colonnes dérivées depuis l’index (datetime index)
df_final_copy['Date ID'] = df_final_copy.index.strftime('%Y%m%d%H')
df_final_copy['Unique ID'] = df_final_copy.index.strftime('%Y%m%d%H%M') + df_final_copy['Station ID']

# Réorganiser les colonnes
df_final_copy = df_final_copy[['Lat', 'Lon'] + [col for col in df_final_copy.columns if col not in ['Lat', 'Lon']]]


In [47]:
df_final.index.name

'Datetime'

In [48]:
df_final.columns

Index(['Lat', 'Lon', 'Wind Direction (°)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Water T° (°C)', 'Water Depth (m)',
       'Station ID', 'Station Zone', 'Sea Temperature Depth (m)',
       'Barometer Elevation (m)', 'Air T° Height (m)', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
       'Sea Level Pressure (hPa)', 'Cloud Cover (%)', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)',
       'Wind Speed (10m)', 'Year', 'Month', 'DayOfWeek', 'DayPeriod'],
      dtype='object')

In [82]:
#imprimer toutes les colonnes numeriquesq
num_cols = df_final.select_dtypes(include=[np.number]).columns

#imprimer toutes les colonnes non-numeriques

non_num_cols = df_final.select_dtypes(exclude=[np.number]).columns
non_num_cols

Index(['Lat', 'Lon', 'Station ID', 'Station Zone', 'Sea Temperature Depth (m)',
       'Barometer Elevation (m)', 'Air T° Height (m)', 'Year', 'Month',
       'DayOfWeek', 'DayPeriod'],
      dtype='object')

In [49]:
display_row_values(df_final)

Datetime             |  Lat    |  Lon      |  Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T° (°C)  |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation (m)  |  Air T° Height (m)  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Cloud Cover (%)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Year  |  Month  |  DayOfWeek  |  DayPeriod
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [50]:
##################################### DimStation ##################################################################################################

df_station = df_final_copy[[
    'Station ID', 'Station Zone', 'Lat', 'Lon'
]].drop_duplicates().set_index('Station ID')

###################################### DimTime ##################################################################################################

df_time = df_final_copy.copy()
df_time = df_time[['Datetime', 'Year', 'Month', 'DayOfWeek', 'DayPeriod']].drop_duplicates()
df_time.set_index('Datetime', inplace=True)

########################################## Facts Meteo #########################################################################

df_facts_meteo = df_final_copy[[
    'Unique ID', # PK
    'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
    'Sea Level Pressure (hPa)', 'Low Clouds (%)', 'Middle Clouds (%)', 'High Clouds (%)',
    "Cloud Cover (%)", 'Visibility (km)', 'Wind Speed (10m)', 'Wind Direction (°)',
    'Wind Gusts (km/h)', 'Barometer Elevation (m)', 'Air T° Height (m)',
    
    'Station ID', # FK
    'Datetime' # FK
]].copy().set_index('Unique ID')

########################################## Facts Ocean #########################################################################

df_facts_ocean = df_final_copy[[
    'Unique ID', # PK
    'Wave Height (m)', 'Average Wave Period (s)', 'Dominant Wave Direction (°)',
    'Water T° (°C)', 'Water Depth (m)', 'Sea Temperature Depth (m)',
    
    'Station ID', # FK
    'Datetime'  # FK
]].copy().set_index('Unique ID')


In [51]:
display_row_values(df_time)

Datetime          |  Year  |  Month  |  DayOfWeek  |  DayPeriod
-----------------------------------------------------------
2025-04-16 17:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 16:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 15:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 14:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 13:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 12:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 11:00  |  2025  |  April  |  Wednesday  |  Morning  
2025-04-16 10:00  |  2025  |  April  |  Wednesday  |  Morning  
2025-04-16 09:00  |  2025  |  April  |  Wednesday  |  Morning  
2025-04-16 08:00  |  2025  |  April  |  Wednesday  |  Morning  


In [52]:
display_row_values(df_station)

Station ID  |  Station Zone                              |  Lat    |  Lon    
--------------------------------------------------------------------------
41008       |  grays reef                                |  31.4   |  -80.87 
41044       |  ne st martin                              |  21.58  |  -58.63 
41049       |  south bermuda                             |  27.5   |  -62.27 
42001       |  mid gulf                                  |  25.93  |  -89.66 
42002       |  west gulf                                 |  25.95  |  -93.78 
42020       |  corpus christi, tx                        |  26.97  |  -96.68 
42036       |  west tampa                                |  28.5   |  -84.5  
42056       |  yucatan basin                             |  19.82  |  -84.98 
42058       |  central caribbean                         |  14.51  |  -75.15 
44007       |  portland                                  |  43.52  |  -70.14 


In [53]:
display_row_values(df_facts_meteo)

Unique ID          |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Cloud Cover (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Wind Direction (°)  |  Wind Gusts (km/h)  |  Barometer Elevation (m)  |  Air T° Height (m)  |  Station ID  |  Datetime        
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
20250416170041008  |  17.5    |  52.0                   |  8.3             |  0.0                  |  1019.2                    |  0.0             |  0.0                |  0.0              |  0.0              |  37.8             |  16.92             |  10.0

In [54]:
display_row_values(df_facts_ocean)

Unique ID          |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T° (°C)  |  Water Depth (m)  |  Sea Temperature Depth (m)  |  Station ID  |  Datetime        
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
20250416170041008  |  0.9              |  4.6                      |  109.0                        |  20.4           |  16.0             |  2.0                        |  41008       |  2025-04-16 17:00
20250416160041008  |  0.9              |  4.6                      |  109.0                        |  20.3           |  16.0             |  2.0                        |  41008       |  2025-04-16 16:00
20250416150041008  |  0.9              |  4.6                      |  109.0                        |  20.3           |  16.0             |  2.0                        |  41008       |  2025-04-16 15:0

In [55]:
# Vérifier les doublons
print(f"Doublons dans df_station : {df_station.duplicated().sum()}")
print(f"Doublons dans df_time : {df_time.duplicated().sum()}")
print(f"Doublons dans df_facts_meteo : {df_facts_meteo.duplicated().sum()}")
print(f"Doublons dans df_facts_ocean : {df_facts_ocean.duplicated().sum()}")

# Vérification des valeurs de base
print(f"\nDescription des données de df_station : \n{df_station.describe()}")
print(f"\nDescription des données de df_time : \n{df_time.describe()}")
print(f"\nDescription des données de df_facts_meteo : \n{df_facts_meteo.describe()}")
print(f"\nDescription des données de df_facts_ocean : \n{df_facts_ocean.describe()}")

Doublons dans df_station : 0
Doublons dans df_time : 1042
Doublons dans df_facts_meteo : 0
Doublons dans df_facts_ocean : 0

Description des données de df_station : 
             Lat         Lon
count  25.000000   25.000000
mean   34.454400 -109.830000
std    12.080752   34.486436
min    14.510000 -172.150000
25%    25.930000 -136.040000
50%    32.500000 -118.050000
75%    41.840000  -80.870000
max    56.610000  -58.630000

Description des données de df_time : 
         Year
count  1098.0
mean   2025.0
std       0.0
min    2025.0
25%    2025.0
50%    2025.0
75%    2025.0
max    2025.0

Description des données de df_facts_meteo : 
             T°(C°)  Relative Humidity (%)  Dew Point (°C)  \
count  26879.000000           26879.000000    26879.000000   
mean      14.975512              79.343727       11.750783   
std        8.357928              11.069979        8.499382   
min      -10.930000              25.000000      -17.900000   
25%        7.520000              73.000000        4.

Checking dim_station DataFrame

In [56]:
# check unique values
print(f"{df_station.shape[0]}\n\n{df_station.nunique()}")
display_row_values(df_station)

25

Station Zone    25
Lat             25
Lon             25
dtype: int64
Station ID  |  Station Zone                              |  Lat    |  Lon    
--------------------------------------------------------------------------
41008       |  grays reef                                |  31.4   |  -80.87 
41044       |  ne st martin                              |  21.58  |  -58.63 
41049       |  south bermuda                             |  27.5   |  -62.27 
42001       |  mid gulf                                  |  25.93  |  -89.66 
42002       |  west gulf                                 |  25.95  |  -93.78 
42020       |  corpus christi, tx                        |  26.97  |  -96.68 
42036       |  west tampa                                |  28.5   |  -84.5  
42056       |  yucatan basin                             |  19.82  |  -84.98 
42058       |  central caribbean                         |  14.51  |  -75.15 
44007       |  portland                                  |  43.52  |  -

Création de la table dim_station et insertion des données

In [57]:
try:
    create_table_in_mysql(df=df_station, engine=engine, table_name=table_dim_station)
    insert_new_rows(df=df_station, engine=engine, table_name=table_dim_station, ref='index')
except Exception as e:
    print(e)

✅ Table 'dim_station' créée avec succès via SQLAlchemy.
📋 Récupération de la table 'dim_station' dans la base de données...
🔎 Vérification des valeurs uniques de l'index...
✅ 0 références existantes trouvées.
🚀 Insertion de 25 nouvelles lignes...
✅ Insertion terminée avec succès.


Checking dim_time DataFrame

In [58]:
print(f"{df_time.shape[0]}\n\n{df_time.nunique()}")
display_row_values(df_time)

1098

Year         1
Month        2
DayOfWeek    7
DayPeriod    4
dtype: int64
Datetime          |  Year  |  Month  |  DayOfWeek  |  DayPeriod
-----------------------------------------------------------
2025-04-16 17:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 16:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 15:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 14:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 13:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 12:00  |  2025  |  April  |  Wednesday  |  Afternoon
2025-04-16 11:00  |  2025  |  April  |  Wednesday  |  Morning  
2025-04-16 10:00  |  2025  |  April  |  Wednesday  |  Morning  
2025-04-16 09:00  |  2025  |  April  |  Wednesday  |  Morning  
2025-04-16 08:00  |  2025  |  April  |  Wednesday  |  Morning  


Création de la table dim_time et insertion des données

In [59]:
try:
    create_table_in_mysql(df=df_time, engine=engine, table_name=table_dim_time)
    insert_new_rows(df=df_time, engine=engine, table_name=table_dim_time, ref='index')
except Exception as e:
    print(e)

✅ Table 'dim_time' créée avec succès via SQLAlchemy.
📋 Récupération de la table 'dim_time' dans la base de données...
🔎 Vérification des valeurs uniques de l'index...
✅ 0 références existantes trouvées.
🚀 Insertion de 1098 nouvelles lignes...
✅ Insertion terminée avec succès.


Checking Meteo Facts DataFrame

In [60]:
print(f"{df_facts_meteo.shape[0]}\n\n{df_facts_meteo.nunique()}")
display_row_values(df_facts_meteo)

26879

T°(C°)                      1820
Relative Humidity (%)         72
Dew Point (°C)               415
Precipitations (mm)           73
Sea Level Pressure (hPa)     566
Low Clouds (%)               101
Middle Clouds (%)            101
High Clouds (%)              101
Cloud Cover (%)              101
Visibility (km)             1262
Wind Speed (10m)            3681
Wind Direction (°)            36
Wind Gusts (km/h)             28
Barometer Elevation (m)        3
Air T° Height (m)              3
Station ID                    25
Datetime                    1098
dtype: int64
Unique ID          |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Cloud Cover (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Wind Direction (°)  |  Wind Gusts (km/h)  |  Barometer Elevation (m)  |  Air T° Height (m)  |  Station ID  |  Datetime        
---------------------------------------

Création de la table facts_meteo et insertion des données

In [61]:
try:
    create_table_in_mysql(df=df_facts_meteo, engine=engine, table_name=table_facts_meteo)
    insert_new_rows(df=df_facts_meteo, engine=engine, table_name=table_facts_meteo, ref='index')
except Exception as e:
    print(e)

✅ Table 'facts_meteo' créée avec succès via SQLAlchemy.
📋 Récupération de la table 'facts_meteo' dans la base de données...
🔎 Vérification des valeurs uniques de l'index...
✅ 0 références existantes trouvées.
🚀 Insertion de 26879 nouvelles lignes...
✅ Insertion terminée avec succès.


Checking Ocean Facts DataFrame

In [62]:
print(f"{df_facts_ocean.shape[0]}\n\n{df_facts_ocean.nunique()}")
display_row_values(df_facts_ocean)

26879

Wave Height (m)                  40
Average Wave Period (s)          61
Dominant Wave Direction (°)     164
Water T° (°C)                   272
Water Depth (m)                  25
Sea Temperature Depth (m)         3
Station ID                       25
Datetime                       1098
dtype: int64
Unique ID          |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T° (°C)  |  Water Depth (m)  |  Sea Temperature Depth (m)  |  Station ID  |  Datetime        
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
20250416170041008  |  0.9              |  4.6                      |  109.0                        |  20.4           |  16.0             |  2.0                        |  41008       |  2025-04-16 17:00
20250416160041008  |  0.9              |  4.6                      |  109.0                   

Création de la table facts_ocean et insertion des données

In [63]:
try:
    create_table_in_mysql(df=df_facts_ocean, engine=engine, table_name=table_facts_ocean)
    insert_new_rows(df=df_facts_ocean, engine=engine, table_name=table_facts_ocean, ref='index')
except Exception as e:
    print(e)

✅ Table 'facts_ocean' créée avec succès via SQLAlchemy.
📋 Récupération de la table 'facts_ocean' dans la base de données...
🔎 Vérification des valeurs uniques de l'index...
✅ 0 références existantes trouvées.
🚀 Insertion de 26879 nouvelles lignes...
✅ Insertion terminée avec succès.


In [ ]:
# try:
#     # Initialiser le metadata
#     metadata = MetaData()
#     metadata.reflect(bind=engine)

#     # Récupérer les tables existantes
#     dim_station = metadata.tables['dim_station']
#     dim_time = metadata.tables['dim_time']
#     facts_meteo = metadata.tables['facts_meteo']
#     facts_ocean = metadata.tables['facts_ocean']

#     # Ajouter les clés étrangères aux tables de faits
#     ForeignKeyConstraint(['Station ID'], [dim_station.c['Station ID']], name='fk_meteo_station').create(facts_meteo, bind=engine)
#     ForeignKeyConstraint(['Datetime'], [dim_time.c['Datetime']], name='fk_meteo_time').create(facts_meteo, bind=engine)

#     ForeignKeyConstraint(['Station ID'], [dim_station.c['Station ID']], name='fk_ocean_station').create(facts_ocean, bind=engine)
#     ForeignKeyConstraint(['Datetime'], [dim_time.c['Datetime']], name='fk_ocean_time').create(facts_ocean, bind=engine)
# except Exception as e:
#     print(str(e))